In [1]:
import os

In [6]:
import string_constants as sc
import reading_util as ru
import data_sample as ds

# read in data

In [7]:
SM_sample = 'qcdSigAll'
qcd_path = os.path.join( sc.concat_result_dir, sc.sample_loc[SM_sample] + sc.concat_result_suffix )
qcd_data_total = ds.read_datasample_from_file(SM_sample, qcd_path)

# load discriminator